In [1]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 472.7/472.7 kB 22.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 16.9 MB/s eta 0:00:00


In [2]:
import tensorflow as tf
import pandas as pd
import numpy as np
import json
import re
import os
import shutil
import datasets

from sklearn.metrics import accuracy_score, f1_score
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from datasets import Dataset


# from transformers import BertTokenizer, TFBertForSequenceClassification
# from transformers import InputExample, InputFeatures
# # from deep_translator import GoogleTranslator
from datasets import Dataset

from transformers import AutoModelForSequenceClassification, AutoTokenizer, Trainer, TrainingArguments
# from sklearn.model_selection import train_test_split
# import torch

# from sklearn.metrics import accuracy_score
# os.environ['TF_ENABLE_ONEDNN_OPTS'] = '0'


In [22]:
from sklearn.model_selection import train_test_split

# Old data


In [3]:
!wget "https://raw.githubusercontent.com/ancatache/LaRoSeDa/main/data_splitted/laroseda_train.json"
f = open('laroseda_train.json')
# returns JSON object as
# a dictionary
train_data = json.load(f)

# Closing file
f.close()

--2024-10-27 02:38:29--  https://raw.githubusercontent.com/ancatache/LaRoSeDa/main/data_splitted/laroseda_train.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4231664 (4.0M) [text/plain]
Saving to: ‘laroseda_train.json’

laroseda_train.json 100%[===================>]   4.04M  --.-KB/s    in 0.01s   

2024-10-27 02:38:30 (287 MB/s) - ‘laroseda_train.json’ saved [4231664/4231664]



In [4]:
!wget "https://raw.githubusercontent.com/ancatache/LaRoSeDa/main/data_splitted/laroseda_test.json"
f = open('laroseda_test.json')
# returns JSON object as
# a dictionary
test_data = json.load(f)

# Closing file
f.close()


--2024-10-27 02:38:30--  https://raw.githubusercontent.com/ancatache/LaRoSeDa/main/data_splitted/laroseda_test.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1025519 (1001K) [text/plain]
Saving to: ‘laroseda_test.json’

laroseda_test.json  100%[===================>]   1001K  --.-KB/s    in 0.006s  

2024-10-27 02:38:30 (163 MB/s) - ‘laroseda_test.json’ saved [1025519/1025519]



In [5]:
def prep_doc(doc):
    cleaned = list()
    for i in doc:
        rev = i['title'] + ' . ' + i['content']
        # rev=i['content']
        # rev = re.sub('(?<! )(?=[.,!?()])|(?<=[.,!?()])(?! )', r' ', rev)
        rev = rev.lower()
        rev = re.sub(r'\s+', ' ', rev)
        rev = re.sub(r'(?<! )(?=[.,!?()"\'/:;/\\\[^\]])|(?<=[.,!?()"\'/:;/\\\[^\]])(?! )', r' ', rev)
        cleaned.append(rev)
    return cleaned


def read_data(set):
    pos_data = list()
    neg_data = list()
    for i in set:
        # print(i)
        match i['starRating']:
            case '1':
                neg_data.append(i)
            case '2':
                neg_data.append(i)
            case '4':
                pos_data.append(i)
            case '5':
                pos_data.append(i)
    return neg_data, pos_data


def read_docs():
    f = open('laroseda_train.json')
    # returns JSON object as
    # a dictionary
    train_data = json.load(f)
    f.close()

    f = open('laroseda_test.json')
    # returns JSON object as
    # a dictionary
    test_data = json.load(f)
    # Closing file
    f.close()

    train_neg, train_pos = read_data(train_data['reviews'])
    test_neg, test_pos = read_data(test_data['reviews'])

    train_pos_clean = prep_doc(train_pos)
    train_neg_clean = prep_doc(train_neg)
    test_pos_clean = prep_doc(test_pos)
    test_neg_clean = prep_doc(test_neg)

    return train_pos_clean, train_neg_clean, test_pos_clean, test_neg_clean

In [ ]:
# Load and preprocess data
train_pos_clean, train_neg_clean, test_pos_clean, test_neg_clean = read_docs()

train_texts = train_pos_clean + train_neg_clean
train_labels = [1] * len(train_pos_clean) + [0] * len(train_neg_clean)

test_texts = test_pos_clean + test_neg_clean
test_labels = [1] * len(test_pos_clean) + [0] * len(test_neg_clean)

In [6]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

# New Data

In [18]:
!wget 'https://raw.githubusercontent.com/ACS-SII/proiect-2024-sentiment-analysis-study/refs/heads/main/new_set/reviews_rating_1.json'
f = open('reviews_rating_1.json')
test_data_1 = json.load(f)
f.close()
!wget 'https://raw.githubusercontent.com/ACS-SII/proiect-2024-sentiment-analysis-study/refs/heads/main/new_set/reviews_rating_2.json'
f = open('reviews_rating_2.json')
test_data_2 = json.load(f)
f.close()
!wget 'https://raw.githubusercontent.com/ACS-SII/proiect-2024-sentiment-analysis-study/refs/heads/main/new_set/reviews_rating_3.json'
f = open('reviews_rating_3.json')
test_data_3 = json.load(f)
f.close()
!wget 'https://raw.githubusercontent.com/ACS-SII/proiect-2024-sentiment-analysis-study/refs/heads/main/new_set/reviews_rating_4.json'
f = open('reviews_rating_4.json')
test_data_4 = json.load(f)
f.close()
!wget 'https://raw.githubusercontent.com/ACS-SII/proiect-2024-sentiment-analysis-study/refs/heads/main/new_set/reviews_rating_5.json'
f = open('reviews_rating_5.json')
test_data_5 = json.load(f)
f.close()


--2024-10-27 02:43:34--  https://raw.githubusercontent.com/ACS-SII/proiect-2024-sentiment-analysis-study/refs/heads/main/new_set/reviews_rating_1.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 588174 (574K) [text/plain]
Saving to: ‘reviews_rating_1.json.1’

reviews_rating_1.js 100%[===================>] 574.39K  --.-KB/s    in 0.004s  

2024-10-27 02:43:34 (137 MB/s) - ‘reviews_rating_1.json.1’ saved [588174/588174]

--2024-10-27 02:43:34--  https://raw.githubusercontent.com/ACS-SII/proiect-2024-sentiment-analysis-study/refs/heads/main/new_set/reviews_rating_2.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|

In [16]:
def prep_doc_new(doc):
    cleaned = list()
    for rev in doc:
        if not isinstance(rev,str):
          continue
        # rev = i['title'] + ' . ' + i['content']
        # rev=i['content']
        # rev = re.sub('(?<! )(?=[.,!?()])|(?<=[.,!?()])(?! )', r' ', rev)
        rev = rev.lower()
        rev = re.sub(r'\s+', ' ', rev)
        rev = re.sub(r'(?<! )(?=[.,!?()"\'/:;/\\\[^\]])|(?<=[.,!?()"\'/:;/\\\[^\]])(?! )', r' ', rev)
        cleaned.append(rev)
    return cleaned


In [23]:
# test_data_pos = test_data_1 + test_data_2 + test_data_4 + test_data_5
data_clean_neg = prep_doc_new(test_data_1+test_data_2+test_data_3)
data_clean_pos = prep_doc_new(test_data_4+test_data_5)

data_clean_pos = data_clean_pos[:10000]
data_clean_neg = data_clean_neg[:10000]

y_opt = [1] * len(data_clean_pos) + [0] * len(data_clean_neg)
X_opt = data_clean_pos + data_clean_neg

print(len(data_clean_neg))
print(len(data_clean_pos))

train_texts, test_texts, train_labels, test_labels = train_test_split(X_opt, y_opt, test_size=0.2, random_state=42)

10000
10000


# Test

In [27]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    if isinstance(logits, tuple):
        logits = logits[0]  # Unpack the first element of the tuple
    print("Logits shape:", logits.shape)
    predictions = np.argmax(logits, axis=-1)
    accuracy = accuracy_score(labels, predictions)
    f1 = f1_score(labels, predictions, average='weighted')
    return {"accuracy": accuracy, "f1_score": f1}

def run_test(model_identifier, train_texts, test_texts, train_labels, test_labels):
    # Load pre-trained model and tokenizer
    tokenizer = AutoTokenizer.from_pretrained(model_identifier)

    # Add a new padding token if not already present
    if tokenizer.pad_token is None:
        tokenizer.add_special_tokens({'pad_token': '[PAD]'})

    model = AutoModelForSequenceClassification.from_pretrained(model_identifier, num_labels=2)

    # Resize token embeddings after adding new special tokens
    model.resize_token_embeddings(len(tokenizer))

    # Set the padding token ID in the model configuration
    model.config.pad_token_id = tokenizer.pad_token_id

    # Tokenize the texts with padding and truncation
    train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=200)
    test_encodings = tokenizer(test_texts, truncation=True, padding=True, max_length=200)

    train_dataset = Dataset.from_dict({
        'input_ids': train_encodings['input_ids'],
        'attention_mask': train_encodings['attention_mask'],
        'label': train_labels
    })

    test_dataset = Dataset.from_dict({
        'input_ids': test_encodings['input_ids'],
        'attention_mask': test_encodings['attention_mask'],
        'label': test_labels
    })

    # Set the format of the datasets for PyTorch
    train_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])
    test_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])

    training_args = TrainingArguments(
        output_dir="./results",
        save_strategy="no",
        num_train_epochs=3,
        per_device_train_batch_size=8,
        per_device_eval_batch_size=32,
        evaluation_strategy="epoch",
        logging_dir="./logs",
        report_to="none"  # Disable wandb

    )

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=test_dataset,
        tokenizer=tokenizer,
        compute_metrics=compute_metrics
    )

    trainer.train()

    # Evaluate the model
    eval_result = trainer.evaluate()
    print(f"Evaluation results: {eval_result}")

    return eval_result



In [ ]:



names = [
        #  'dumitrescustefan/t5-v1_1-base-romanian'
        #  'racai/distilbert-base-romanian-uncased', #--
        #  'racai/distilbert-base-romanian-cased', #--
         'dumitrescustefan/bert-base-romanian-uncased-v1', #---
        #  'dumitrescustefan/bert-base-romanian-cased-v1', #---
        #  'xlm-roberta-base' #---
        #  'bert-base-multilingual-cased',  #---
        #  'bert-base-multilingual-uncased',  #---
        #  'gpt2',  # The standard GPT-2 model identifier #---
        #  'DGurgurov/xlm-r_romanian_sentiment' #---
        #  'readerbench/RoGPT2-base'

]

 #  'OpenLLM-Ro/RoLlama2-7b-Base',
 #  'OpenLLM-Ro/RoGemma-7b-Instruct',

# Run the test with GPT-2
eval_result = run_test(names[0], train_texts, test_texts, train_labels, test_labels)


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at dumitrescustefan/bert-base-romanian-uncased-v1 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy,F1 Score
1,0.484900,0.471766,0.785250,0.782627
2,0.370200,0.526162,0.793000,0.792883


Logits shape: (4000, 2)
Logits shape: (4000, 2)


In [ ]:
breq

In [ ]:
torch.cuda.empty_cache()


In [ ]:
from google.colab import runtime
runtime.unassign()